# Action recognition using TAO ActionRecognitionNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## Sample prediction of ActionRecognitionNet
<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/action_recognition/ARNet_inference.png?raw=true" width="960">

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train 3D RGB only for action recognition on the subset of [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .etlt file for deployment to DeepStream.

At the end of this notebook, you will have a trained and optimized `action_recognition` model that you
may deploy via [DeepStream](https://developer.nvidia.com/deepstream-sdk).

## Table of Contents

This notebook shows an example usecase of ActionRecognitionNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
7. [Deploy](#head-7)


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
import os

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=/path/to/local/tao-experiments

os.environ["HOST_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data", "actionrecognitionnet")
os.environ["HOST_RESULTS_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "actionrecognitionnet")

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=/path/to/local/tao-experiments/action_recognition_net
# The sample spec files are present in the same path as the downloaded samples.
os.environ["HOST_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

# Set your encryption key, and use the same key for all commands
%env KEY = nvidia_tao

In [ ]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       # Mapping the data directory
       {
           "source": os.environ["LOCAL_PROJECT_DIR"],
           "destination": "/workspace/tao-experiments"
       },
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       },
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.


In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

 We will be using the [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset for the tutorial. Download the HMDB51 dataset and unrar them firstly (We choose fall_floor/ride_bike for this tutorial): 

In [ ]:
# install unrar
# NOTE: The following commands require `sudo`. You can run the command outside the notebook.
!apt update
!apt-get install unrar

In [ ]:
# download the dataset and unrar the files
!wget --no-check-certificate -P $HOST_DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
!mkdir -p $HOST_DATA_DIR/videos && unrar x -o+ $HOST_DATA_DIR/hmdb51_org.rar $HOST_DATA_DIR/videos
!mkdir -p $HOST_DATA_DIR/raw_data
!unrar x -o+ $HOST_DATA_DIR/videos/fall_floor.rar $HOST_DATA_DIR/raw_data
!unrar x -o+ $HOST_DATA_DIR/videos/ride_bike.rar $HOST_DATA_DIR/raw_data

Clone the dataset process script

In [ ]:
!if [ -d tao_toolkit_recipes ]; then rm -rf tao_toolkit_recipes; fi
!git clone https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes

Install the dependency for data generator:

In [ ]:
!pip3 install xmltodict opencv-python

Run the process script. 

In [ ]:
!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB_RGB.sh $HOST_DATA_DIR/raw_data $HOST_DATA_DIR/processed_data

We also provide scripts to preprocess optical flow dataset. The following cells for processing optical flow dataset is `Optional`.

`OPTIONAL:` Download the app based on NVOF SDK to generate optical flow. It is packaged with this notebook.

In [ ]:
#!echo <passwd> | sudo -S apt install -y libfreeimage-dev

`OPTIONAL` Run the process script for HMDB. 

`IMPORTANT NOTE`: to run the `preprocess_HMDB.sh` generating optical flow, a Turing or Ampere above GPU is needed. 

In [ ]:
#!cp ./AppOFCuda tao_toolkit_recipes/tao_action_recognition/data_generation/
#!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB.sh $HOST_DATA_DIR/raw_data $HOST_DATA_DIR/processed_data

In [ ]:
# download the split files and unrar
!wget --no-check-certificate -P $HOST_DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
!mkdir -p $HOST_DATA_DIR/splits && unrar x -o+ $HOST_DATA_DIR/test_train_splits.rar $HOST_DATA_DIR/splits

In [ ]:
# run split_HMDB to generate training split
!if [ -d $HOST_DATA_DIR/train ]; then rm -rf $HOST_DATA_DIR/train $HOST_DATA_DIR/test; fi
!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && python3 ./split_dataset.py $HOST_DATA_DIR/processed_data $HOST_DATA_DIR/splits/testTrainMulti_7030_splits $HOST_DATA_DIR/train  $HOST_DATA_DIR/test

In [ ]:
# verify
!ls -l $HOST_DATA_DIR/train
!ls -l $HOST_DATA_DIR/train/ride_bike
!ls -l $HOST_DATA_DIR/test
!ls -l $HOST_DATA_DIR/test/ride_bike

### 2.2 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
import os
%env CLI=ngccli_cat_linux.zip
!mkdir -p $HOST_RESULTS_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $HOST_RESULTS_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $HOST_RESULTS_DIR/ngccli
!unzip -u "$HOST_RESULTS_DIR/ngccli/$CLI" -d $HOST_RESULTS_DIR/ngccli/
!rm $HOST_RESULTS_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("HOST_RESULTS_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/actionrecognitionnet:*

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/pretrained

In [ ]:
# Pull pretrained model from NGC 
!ngc registry model download-version "nvidia/tao/actionrecognitionnet:trainable_v1.0" --dest $HOST_RESULTS_DIR/pretrained
# Download the optical flow model from NGC
# !ngc registry model download-version "nvidia/tao/actionrecognitionnet:trainable_v2.0" --dest $HOST_RESULTS_DIR/pretrained

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v1.0

## 3. Provide training specification <a class="anchor" id="head-2"></a>

We provide specification files to configure the training parameters including:

* model: configure the model setting
    * model_type: type of model, rgb/of/joint
    * backbone: resnet18/34/50/101/152 
    * rgb_seq_length: length of RGB input sequence
    * input_type: 2d/3d
    * sample_strategy: consecutive
    * dropout_ratio: probability to drop the hidden units
* train: configure the training hyperparameters
    * optim_config
    * epochs
    * checkpoint_interval
* dataset: configure the dataset and augmentation methods
    * train_dataset_dir
    * val_dataset_dir
    * label_map: map the class label to id
    * output_shape
    * batch_size
    * workers: number of workers to do data loading
    * clips_per_video: number of clips to be sampled from single video
    * augmentation_config

Please refer to the TAO documentation about ActionRecognitionNet to get all the parameters that are configurable.

In [ ]:
!cat $HOST_SPECS_DIR/experiment_rgb_3d_finetune.yaml

## 4. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.

# The data is saved here
%env DATA_DIR = /data
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results

### 4.1 Train 3D model:

We provide pretrained RGB-only model trained on HMDB5 dataset. With the pretrained model, we can even get better accuracy with less epochs.

`KNOWN ISSUE`: 
- 1) The training log will be corrupted by pytorch warning in the notebook:

     `[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)` 
     
     To see the full log in std out, please run the command in terminal. 
- 2) "=" in the checkpoint file name should removed before using the checkpoint in command.

In [ ]:
print("Train RGB only model with PTM")
!tao model action_recognition train \
                  -e $SPECS_DIR/experiment_rgb_3d_finetune.yaml \
                  encryption_key=$KEY \
                  results_dir=$RESULTS_DIR/rgb_3d_ptm \
                  model.rgb_pretrained_model_path=$RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v1.0/resnet18_3d_rgb_hmdb5_32.tlt  \
                  model.rgb_pretrained_num_classes=5

In [ ]:
## Training command for multi-gpu training. We can define the number of gpus and specify which GPU's are to be used by setting the `train.gpu_ids` parameter.
## The following command will trigger multi-gpu training on gpu 0 and gpu 1.
# !tao model action_recognition train \
#                   -e $SPECS_DIR/experiment_rgb_3d_finetune.yaml \
#                   encryption_key=$KEY \
#                   train.gpu_ids=[0,1] \
#                   results_dir=$RESULTS_DIR/rgb_3d_ptm \
#                   model.rgb_pretrained_model_path=$RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v1.0/resnet18_3d_rgb_hmdb5_32.tlt  \
#                   model.rgb_pretrained_num_classes=5

In [ ]:
print('Encrypted checkpoints:')
print('---------------------')
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d_ptm/train

In [ ]:
# You can set NUM_EPOCH to the epoch corresponding to any saved checkpoint
# %env NUM_EPOCH=029

# Get the name of the checkpoint corresponding to your set epoch
# tmp=!ls $HOST_RESULTS_DIR/rgb_3d_ptm/train/*.pth | grep epoch_$NUM_EPOCH
# %env CHECKPOINT={tmp[0]}

# Or get the latest checkpoint
os.environ["CHECKPOINT"] = os.path.join(os.getenv("HOST_RESULTS_DIR"), "rgb_3d_ptm/train/ar_model_latest.pth")

print('Rename a trained model: ')
print('---------------------')
!cp $CHECKPOINT $HOST_RESULTS_DIR/rgb_3d_ptm/train/rgb_only_model.tlt
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d_ptm/train/rgb_only_model.tlt

### `OPTIONAL` 4.2 Train optical flow only model

`Important Note` The following cells are using optical flow dataset. 

We will train a 3D OF-only model

In [ ]:
# print("Train 3D OF-only model")
# !tao model action_recognition train \
#                   -e $SPECS_DIR/experiment_of_3d_finetune.yaml \
#                   encryption_key=$KEY \
#                   results_dir=$RESULTS_DIR/of_3d_ptm \
#                   dataset.train_dataset_dir=$DATA_DIR/train \
#                   dataset.val_dataset_dir=$DATA_DIR/test \
#                   model.of_pretrained_model_path=$RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v2.0/resnet18_3d_of_hmdb5_32_a100.tlt  \
#                   model.of_pretrained_num_classes=5

In [ ]:
# print("To resume training from a checkpoint, set the resume_training_checkpoint_path option to be the .tlt you want to resume from")
# print("remember to remove the `=` in the checkpoint's file name")
# !tao model action_recognition train \
#                   -e $SPECS_DIR/experiment_of_3d_finetune.yaml \
#                   encryption_key=$KEY \
#                   results_dir=$RESULTS_DIR/of_3d_ptm \
#                   train.resume_training_checkpoint_path=

In [ ]:
# print('Encrypted checkpoints:')
# print('---------------------')
# !ls -ltrh $HOST_RESULTS_DIR/of_3d_ptm/train

In [ ]:
# You can set NUM_EPOCH to the epoch corresponding to any saved checkpoint
# %env NUM_EPOCH=029

# Get the name of the checkpoint corresponding to your set epoch
# tmp=!ls $HOST_RESULTS_DIR/of_3d_ptm/train/*.pth | grep epoch_$NUM_EPOCH
# %env CHECKPOINT={tmp[0]}

# Or get the latest checkpoint
# os.environ["CHECKPOINT"] = os.path.join(os.getenv("HOST_RESULTS_DIR"), "of_3d_ptm/train/ar_model_latest.pth")

# print('Rename a trained model: ')
# print('---------------------')
# !cp $CHECKPOINT $HOST_RESULTS_DIR/of_3d_ptm/train/of_only_model.tlt
# !ls -ltrh $HOST_RESULTS_DIR/of_3d_ptm/train/of_only_model.tlt

## 5. Evaluate trained models <a class="anchor" id="head-4"></a>

We provide two different sample strategy to evaluate the pretrained model on video clips.

* `center` mode: pick up the middle frames of a sequence to do inference. For example, if the model requires 32 frames as input and a video clip has 128 frames, then we will choose the frames from index 48 to index 79 to do the inference. 
* `conv` mode: convolutionly sample 10 sequences out of a single video and do inference. The final results are averaged.

Evaluate RGB model trained with PTM

In [ ]:
!tao model action_recognition evaluate \
                    -e $SPECS_DIR/experiment_rgb_3d_finetune.yaml \
                    encryption_key=$KEY \
                    results_dir=$RESULTS_DIR/rgb_3d_ptm \
                    dataset.workers=0 \
                    evaluate.checkpoint=$RESULTS_DIR/rgb_3d_ptm/train/rgb_only_model.tlt  \
                    evaluate.batch_size=1 \
                    evaluate.test_dataset_dir=$DATA_DIR/test \
                    evaluate.video_eval_mode=center

`Optional:` Evaluate OF model

In [ ]:
# !tao model action_recognition evaluate \
#                     -e $SPECS_DIR/experiment_of_3d_finetune.yaml \
#                     encryption_key=$KEY \
#                     results_dir=$RESULTS_DIR/of_3d_ptm \
#                     dataset.workers=0 \
#                     evaluate.checkpoint=$RESULTS_DIR/of_3d_ptm/train/of_only_model.tlt  \
#                     evaluate.batch_size=1 \
#                     evaluate.test_dataset_dir=$DATA_DIR/test \
#                     evaluate.video_eval_mode=center

## 6. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the action recognition inference tool to generate inferences with the trained RGB models and print the results. 

There are also two modes for inference just like evaluation: `center` mode and `conv` mode. And the final output will show each input sequence label in the videos like:
`[video_sample_path] [labels list for sequences in the video sample]`

In [ ]:
!tao model action_recognition inference \
                    -e $SPECS_DIR/experiment_rgb_3d_finetune.yaml \
                    encryption_key=$KEY \
                    results_dir=$RESULTS_DIR/rgb_3d_ptm \
                    dataset.workers=0 \
                    inference.checkpoint=$RESULTS_DIR/rgb_3d_ptm/train/rgb_only_model.tlt \
                    inference.inference_dataset_dir=$DATA_DIR/test/ride_bike \
                    inference.video_inf_mode=center

`Optional:` Inference with OF-only model

In [ ]:
# !tao model action_recognition inference \
#                     -e $SPECS_DIR/experiment_of_3d_finetune.yaml \
#                     encryption_key=$KEY \
#                     results_dir=$RESULTS_DIR/of_3d_ptm \
#                     dataset.workers=0 \
#                     inference.checkpoint=$RESULTS_DIR/of_3d_ptm/train/of_only_model.tlt \
#                     inference.inference_dataset_dir=$DATA_DIR/test/ride_bike \
#                     inference.video_inf_mode=center

## 7. Deploy! <a class="anchor" id="head-6"></a>

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/export

In [ ]:
# Export the RGB model to encrypted ONNX model
!tao model action_recognition export \
                   -e $SPECS_DIR/experiment_rgb_3d_finetune.yaml \
                   encryption_key=$KEY \
                   results_dir=$RESULTS_DIR/rgb_3d_ptm \
                   export.checkpoint=$RESULTS_DIR/rgb_3d_ptm/train/rgb_only_model.tlt \
                   export.onnx_file=$RESULTS_DIR/export/rgb_resnet18_3.onnx

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/export

This notebook has come to an end. You may continue by deploying this RGB model to [DeepStream](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_3D_Action.html)

`Optional` Export OF-Only model

In [ ]:
# # Export the OF model to encrypted ONNX model
# !tao model action_recognition export \
#                    -e $SPECS_DIR/experiment_of_3d_finetune.yaml \
#                    encryption_key=$KEY \
#                    results_dir=$RESULTS_DIR/of_3d_ptm \
#                    export.checkpoint=$RESULTS_DIR/of_3d_ptm/of_only_model.tlt\
#                    export.onnx_file=$RESULTS_DIR/export/of_resnet18_3.onnx

OF model is not supported in DeepStream. But you can play with stand-alone TensorRT inference in [tao_toolkit_recipes](https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes/tree/main/tao_action_recognition/tensorrt_inference) 